# Initial Analysis to see if foreign success predictions will work

In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [97]:


################## Make it this way (the fast way) for the blog post
# load data into list of lists and then put it into pandas
##################

allFiles = glob.glob("/Users/williamcosby/Documents/metis/Project_Luther/movie_*.csv")
movie_data = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)
movie_data_raw = pd.concat(list_)

CParserError: Error tokenizing data. C error: Expected 6 fields in line 645, saw 7


In [61]:
# movie_data_2015_2016 = pd.read_csv('movie_data_2015_2016.csv')

In [62]:
movie_data_raw.head()

,MOVIE_NAME,DOMESTIC_OPENING,FOREIGN_TOTAL,BUDGET,GENRE,RELEASE_DATE
0,Marvel's The Avengers,207438708,896200000,220000000,Action / Adventure,2012-05-04 00:00:00
1,The Dark Knight Rises,160887295,636800000,250000000,Action Thriller,2012-07-20 00:00:00
2,The Hunger Games,152535747,286384032,78000000,Action / Adventure,2012-03-23 00:00:00
3,Skyfall,88364714,804200736,200000000,Action,2012-11-09 00:00:00
4,The Hobbit:An Unexpected Journey,84617303,718100000,0,Fantasy,2012-12-14 00:00:00


# Find number of movies that have foreign gross info

In [63]:
# total number of movies
movie_data_raw.shape

(4256, 6)

In [64]:
# movies with foreign earnings information
movie_data_raw[movie_data_raw['FOREIGN_TOTAL']!=0].shape

(1528, 6)

# Subset to get just the movies with foreign earnings information

In [65]:
movie_data = movie_data_raw[movie_data_raw['FOREIGN_TOTAL']!=0]

In [72]:
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1528 entries, 0 to 970
Data columns (total 64 columns):
MOVIE_NAME                  1528 non-null object
DOMESTIC_OPENING            1528 non-null int64
FOREIGN_TOTAL               1528 non-null int64
BUDGET                      1528 non-null int64
RELEASE_DATE                1528 non-null object
GENRE_Action                1528 non-null float64
GENRE_Action / Adventure    1528 non-null float64
GENRE_Action / Crime        1528 non-null float64
GENRE_Action Comedy         1528 non-null float64
GENRE_Action Drama          1528 non-null float64
GENRE_Action Fantasy        1528 non-null float64
GENRE_Action Horror         1528 non-null float64
GENRE_Action Thriller       1528 non-null float64
GENRE_Adventure             1528 non-null float64
GENRE_Adventure Comedy      1528 non-null float64
GENRE_Animation             1528 non-null float64
GENRE_Comedy                1528 non-null float64
GENRE_Comedy / Drama        1528 non-null float64
GE

# Start some analysis

### Need to turn genre into a one-hot encoding

In [68]:
movie_data = pd.get_dummies(movie_data,columns=["GENRE"])

In [69]:
movie_data.head()

,MOVIE_NAME,DOMESTIC_OPENING,FOREIGN_TOTAL,BUDGET,RELEASE_DATE,GENRE_Action,GENRE_Action / Adventure,GENRE_Action / Crime,GENRE_Action Comedy,GENRE_Action Drama,...,GENRE_Sci-Fi Fantasy,GENRE_Sci-Fi Horror,GENRE_Sci-Fi Thriller,GENRE_Sports Comedy,GENRE_Sports Drama,GENRE_Thriller,GENRE_War,GENRE_War Drama,GENRE_Western,GENRE_Western Comedy
0,Marvel's The Avengers,207438708,896200000,220000000,2012-05-04 00:00:00,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,The Dark Knight Rises,160887295,636800000,250000000,2012-07-20 00:00:00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,The Hunger Games,152535747,286384032,78000000,2012-03-23 00:00:00,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Skyfall,88364714,804200736,200000000,2012-11-09 00:00:00,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,The Hobbit:An Unexpected Journey,84617303,718100000,0,2012-12-14 00:00:00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Will want to encode the dates as 1st quarter, 2nd quarter, 3rd quarter, 4th quarter
#### Do this by first transforming the column into strings "1","2","3","4", then apply pandas .get_dummies to make it one-hot

In [70]:
from pandas import DatetimeIndex
import dateutil.parser

In [76]:
# turn RELEASE_DATE into actual datetime
movie_data['RELEASE_DATE'] = movie_data['RELEASE_DATE'].apply(lambda x: dateutil.parser.parse(x))

In [77]:
movie_data['RELEASE_DATE'].dtype

dtype('<M8[ns]')

In [79]:
movie_data["RELEASE_QUARTER"] = movie_data["RELEASE_DATE"].apply(lambda x: x.quarter)

In [90]:
# now just want release quarter
movie_data.drop(['RELEASE_DATE'],axis=1,inplace=True)

#### Now vectorize release quarter column

In [83]:
movie_data = pd.get_dummies(movie_data,columns=["RELEASE_QUARTER"])

In [91]:
movie_data.head()

,MOVIE_NAME,DOMESTIC_OPENING,FOREIGN_TOTAL,BUDGET,GENRE_Action,GENRE_Action / Adventure,GENRE_Action / Crime,GENRE_Action Comedy,GENRE_Action Drama,GENRE_Action Fantasy,...,GENRE_Sports Drama,GENRE_Thriller,GENRE_War,GENRE_War Drama,GENRE_Western,GENRE_Western Comedy,RELEASE_QUARTER_1,RELEASE_QUARTER_2,RELEASE_QUARTER_3,RELEASE_QUARTER_4
0,Marvel's The Avengers,207438708,896200000,220000000,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,The Dark Knight Rises,160887295,636800000,250000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,The Hunger Games,152535747,286384032,78000000,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,Skyfall,88364714,804200736,200000000,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,The Hobbit:An Unexpected Journey,84617303,718100000,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Now pop the foreign total column off into its own thing (the label)

In [92]:
X,y = movie_data.drop('FOREIGN_TOTAL', axis=1),movie_data['FOREIGN_TOTAL']

# Check the data for missing values, and other weird things

## Impute what I can into the raw data

### Check Domestic Openings

In [93]:
len(movie_data[movie_data["DOMESTIC_OPENING"] == 0])

30

# Note!!!!!! one reason for weird domest/foreign offset is if a movie is released somewhere like korea...it might do super well there but not well in america (which is considered domestic)

# Have option of ignoring films with foreign as the genre

In [94]:
movies_no_domestic = movie_data[movie_data["DOMESTIC_OPENING"] == 0]

In [95]:
movies_no_domestic

,MOVIE_NAME,DOMESTIC_OPENING,FOREIGN_TOTAL,BUDGET,GENRE_Action,GENRE_Action / Adventure,GENRE_Action / Crime,GENRE_Action Comedy,GENRE_Action Drama,GENRE_Action Fantasy,...,GENRE_Sports Drama,GENRE_Thriller,GENRE_War,GENRE_War Drama,GENRE_Western,GENRE_Western Comedy,RELEASE_QUARTER_1,RELEASE_QUARTER_2,RELEASE_QUARTER_3,RELEASE_QUARTER_4
219,Masquerade (2012),0,79839675,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
225,Meerkats,0,482021,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
251,Delicacy,0,9197620,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
263,The Lady,0,3404191,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
266,A Werewolf Boy,0,41336378,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
349,Iron Sky,0,8012949,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
408,Free Men,0,564293,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
817,Flying Monsters,0,5107988,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
845,RA One,0,33400000,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1415,Sea Rex 3D:Journey to a Prehistoric World,0,9907568,5000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [88]:
import sklearn
from sklearn.cross_validation import train_test_split,KFold,cross_val_score
from pandas.tools.plotting import scatter_matrix

## Want to get scatter plots of variables with eachother and with the label